In [18]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

source = pd.read_csv('SSC_discharge.csv')
target = pd.read_csv('SSC_pooled.csv')


source_data = np.array(source.values[:,1:],dtype=np.float32)
target_data = np.array(target.values[:,1:],dtype=np.float32)

s_mean,s_std = np.mean(source_data,axis=0),np.std(source_data,axis=0)
source_data = (source_data - s_mean) / s_std

mask = pd.isna(target_data)
target_data [mask] = 0
#plt.plot(target_data[:,1])
#print(target_data.shape)

mask_target_data  = np.ma.array(target_data,mask = mask)
t_mean,t_std = np.ma.mean(mask_target_data,axis=0),np.ma.std(mask_target_data,axis=0)
target_data = (target_data - t_mean) / t_std


T = 15
lamd_coef = 0.75
## Source-Dominant data 



## Target-Dominant data 
#print(target_data)
target_data_after = np.zeros(target_data.shape)
target_data_before = np.zeros(target_data.shape)

# after T time steps 
for i in range(T):
    target_data_after += source_data[i+1:i+1+target_data.shape[0],:]
# previous T time steps 
for i in range(T):
    comple_data = np.zeros((i+1,target_data.shape[1]))
    target_data_before += np.concatenate([comple_data,source_data[0:target_data.shape[0]-i-1,:]],axis=0)

#### generate target_dominant data 
target_dominant_data = lamd_coef * target_data + (1-lamd_coef) /(2*T+1) * (target_data_after+target_data_before+source_data[0:target_data.shape[0],:])
target_dominant_data = target_dominant_data * t_std + t_mean
target_dominant_data[mask] = np.nan

print(target_dominant_data.shape)


#plt.plot(target_dominant_data[:,1])

print(target)

target.iloc[:, 1:] = target_dominant_data

print(target)

target.to_csv('SSC_target_dominant.csv',index=None)






(2726, 20)
           date  4178000  4182000  4183000  4183500  4184500  4185000  \
0     2015/4/15      NaN      NaN      NaN      NaN      NaN      NaN   
1     2015/4/16      NaN      NaN      NaN      NaN      NaN      NaN   
2     2015/4/17      NaN      NaN      NaN      NaN      NaN      NaN   
3     2015/4/18      NaN      NaN      NaN      NaN      NaN      NaN   
4     2015/4/19      NaN      NaN      NaN      NaN      NaN      NaN   
...         ...      ...      ...      ...      ...      ...      ...   
2721  2022/9/26     10.0      NaN     17.0      0.6     53.0   27.008   
2722  2022/9/27     10.0      NaN     18.0      0.5      NaN      NaN   
2723  2022/9/28      9.0      NaN     20.0      0.5      NaN      NaN   
2724  2022/9/29      9.0      NaN     21.0      0.5      NaN      NaN   
2725  2022/9/30      8.0     18.0     22.0      0.5      NaN      NaN   

      4185318    4185440  4186500  ...  4188496  4189000  4190000  4191058  \
0          13   4.373200     37.0 